# Guided Project - Explore U.S. Deaths

In [215]:
#reading out data to see the data format
f = open("US_births_1994-2003_CDC_NCHS.csv","r")
data = f.read()
data = data.split("\n")
string_list = data[1:11]  

In [216]:
#function to load files systematically
def read_csv(fileName):
    f = open(fileName,"r")
    data = (f.read()).split("\n")
    string_list = data[1:]
    final_list = []
    
    for each in string_list:
        int_fields = []
        string_fields = each.split(",")
        for data in string_fields:
            data_int = int(data)
            int_fields.append(data_int)
        final_list.append(int_fields)
    return final_list

cdc_list=read_csv("US_births_1994-2003_CDC_NCHS.csv")

# Calculate the totals for months

In [217]:
def month_births(birth_list):
    births_per_month = {}
    for each in birth_list:
        month = each[1]
        births = each[4]
        if month in births_per_month:
            births_per_month[month]=births_per_month[month]+births
        else:
            births_per_month[month]=births
    return births_per_month

cdc_month_births = month_births(cdc_list)

# Calculate the totals for days of week

In [218]:
def dow_births(birth_list):
    birth_per_weekday = {}
    for each in birth_list:
        week = each[3]
        births = each[4]
        if week in birth_per_weekday:
            birth_per_weekday[week] = birth_per_weekday[week]+births
        else:
            birth_per_weekday[week] = births
    return birth_per_weekday
cdc_day_births = dow_births(cdc_list)

# Calculate the totals for different parameters

In [219]:
def calc_counts(data,column):
    birth_dict = {}
    for each in data:
        info_column = each[column]
        births = each[4]
        if info_column in birth_dict:
            birth_dict[info_column] = birth_dict[info_column]+births
        else:
            birth_dict[info_column] = births
            
    return birth_dict
cdc_year_births = calc_counts(cdc_list,0)
cdc_month_births = calc_counts(cdc_list,1)
cdc_dom_births = calc_counts(cdc_list,2)
cdc_dow_births = calc_counts(cdc_list,3)

# Calculate the min and max values for any dictionary that's passed in

In [220]:
def min_and_max(birth_data):
    min_value = 0
    max_value = 0
    min_value = min(birth_data.values())
    max_value = max(birth_data.values())
    return "Min: "+str(min_value)+" and Max: " +str(max_value)
min_and_max(cdc_dow_births)

'Min: 4079723 and Max: 6446196'

# Check the birth growth trend by years

In [221]:
def year_growth_trend(birth_data):
    current_data=0
    past_data=0
    info = []
    text = ""
    info = calc_counts(birth_data,0)
    for year,births in sorted(info.items()):
        current_data = births
        if past_data == 0:
            text= "Birth growth information not available in {}.".format(year)
            past_data = current_data
            paragraph = ""
        else:
            if current_data > past_data:
                text = "Births increased in {}".format(year)
                past_data = current_data
            elif current_data < past_data:
                text = "Births decreased in {}".format(year)
                past_data = current_data
        paragraph = paragraph + "\n" + text
    print(paragraph)
growth = year_growth_trend(cdc_list)


Birth growth information not available in 1994.
Births decreased in 1995
Births decreased in 1996
Births decreased in 1997
Births increased in 1998
Births increased in 1999
Births increased in 2000
Births decreased in 2001
Births decreased in 2002
Births increased in 2003


# Check the growth trend of specific parameter by years. 
For example, "trend" shows how did the number of births on Saturday change each year between 1994 and 2003?
sort_by can be "year","month","day_of_month","day_of_week".
If sort_by="year",query can be any integer

In [222]:
def growth_trend(fileName,sort_by="month",query=3):
    cdc_list = read_csv(fileName)
    count_dict={}
    
    #Data assortment using different parameters
    if sort_by == "year":
        index = 0
    elif sort_by == "month":
        index = 1
    elif sort_by == "day_of_month":
        index = 2 
    elif sort_by == "day_of_week":
        index = 3 
    for each in cdc_list:
        year = each[0]
        if sort_by == "year":
            query = each[index]
        query_field = each[index]
        births = each[4]
        if query_field == query:
            if year in count_dict:
                count_dict[year]=count_dict[year]+births
            else:
                count_dict[year]=births
    print(count_dict)
    
    #Checking for birth growth trend base on the parameter chosen
    past_data = 0
    current_data = 0
    for year,births in sorted(count_dict.items()):
        current_data = births
        if past_data == 0:
            text= "Birth growth information is not available in {}.".format(year)
            past_data = current_data
            paragraph = ""
        elif current_data > past_data:
            text = "Births increased in {}".format(year)
            past_data = current_data
        elif current_data < past_data:
            text = "Births decreased in {}".format(year)
            past_data = current_data
        paragraph = paragraph + "\n" + text    
    print(paragraph)
            
trend = growth_trend("US_births_1994-2003_CDC_NCHS.csv","day_of_week",6)

{1994: 474732, 1995: 459580, 1996: 456261, 1997: 450840, 1998: 453776, 1999: 449985, 2000: 469794, 2001: 453928, 2002: 445770, 2003: 447445}

Birth growth information is not available in 1994.
Births decreased in 1995
Births decreased in 1996
Births decreased in 1997
Births increased in 1998
Births decreased in 1999
Births increased in 2000
Births decreased in 2001
Births decreased in 2002
Births increased in 2003


# Merge the CDC and SSA datasets together
If entries are found to be double entered in both datasets, the births 

In [223]:
def merging_CDC_SSA_data(csc_fileName,ssa_fileName):
    import copy
    cdc_list = read_csv(csc_fileName)   
    ssa_list = read_csv(ssa_fileName)
    cdc_date_list = []
    ssa_date_list = [] 
    merged_data = []
#copy.deepcopy is used to ensure cdc_list is not altered when merged_data is altered
#merged_data = cdc_list[:] can be used if alteration of cdc_list does not matter
    merged_data = copy.deepcopy(cdc_list)
    for each_cdc in cdc_list:
        cdc_date = each_cdc[0:4]
        cdc_date_list.append(cdc_date)
    for each_ssa in ssa_list:
        ssa_date = each_ssa[0:4]
        ssa_date_list.append(ssa_date)       

        if ssa_date in cdc_date_list:
            index_cdc = cdc_date_list.index(ssa_date)
            index_ssa = ssa_date_list.index(ssa_date)
            birth_cdc = cdc_list[index_cdc][4]
            birth_ssa = ssa_list[index_ssa][4]
            birth_avg = 0.5*birth_cdc+0.5*birth_ssa
            merged_data[index_cdc][4]=birth_avg                    
        else:
            index = ssa_date_list.index(ssa_date)
            merged_data.append(ssa_list[index])
    return merged_data
    
merged_data_cdc_ssa = merging_CDC_SSA_data("US_births_1994-2003_CDC_NCHS.csv","US_births_2000-2014_SSA.csv")